In [1]:
import pandas as pd
import re

In [2]:
# Load the dataset
df = pd.read_csv('cleaned_data.csv')
df

,Clean_News_Headline,Clean_Source,Label,News_Headline,Source
0,"{'biden': 1.0, 'joe': 1.0, 'endorsed': 1.0, 'l...","{'trump': 1.0, 'donald': 1.0}",FALSE,6.0,2.0
1,"{'trump': 1.0, 'cnn': 1.0, 'voted': 1.0, 'aire...","{'trump': 1.0, 'donald': 1.0}",pants-fire,16.0,2.0
2,"{'model': 1.0, 'american': 1.0, 'fans': 1.0, '...","{'posts': 1.0, 'facebook': 1.0}",FALSE,11.0,2.0
3,"{'years': 1.0, 'gross': 1.0, 'state': 1.0, 're...","{'junge': 1.0, 'paul': 1.0}",barely-true,18.0,2.0
4,"{'year': 1.0, 'crisis': 1.0, 'racism': 1.0, 'd...","{'kowalik': 1.0, 'jeanette': 1.0}",TRUE,10.0,2.0
...,...,...,...,...,...
9955,"{'communities': 1.0, 'criminals': 1.0, 'danger...","{'mccaul': 1.0, 'michael': 1.0}",barely-true,17.0,2.0
9956,"{'transparent': 1.0, 'court': 1.0, 'surveillan...","{'obama': 1.0, 'barack': 1.0}",pants-fire,5.0,2.0
9957,"{'appropriation': 1.0, 'passed': 1.0, 'senate'...","{'kingston': 1.0, 'jack': 1.0}",mostly-true,6.0,2.0
9958,"{'security': 1.0, 'spanish': 1.0, 'border': 1....","{'rohrabacher': 1.0, 'dana': 1.0}",barely-true,8.0,2.0


In [3]:
# Importing warnings library.
# Using this to avoid the warnings from getting displayed.
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Define a function called Fake_label that takes a text input
def Fake_label(text):
    # Replace "TRUE" with '6' in the text
    a = re.sub("TRUE", '6', text)
    # Replace "mostly-true" with '5' in the text
    b = re.sub("mostly-true", '5', a)
    # Replace "half-true" with '4' in the text
    c = re.sub("half-true", '4', b)
    # Replace "barely-true" with '3' in the text
    d = re.sub("barely-true", '3', c)
    # Replace "FALSE" with '2' in the text
    e = re.sub("FALSE", '2', d)
    # Replace "full-flop" with '3' in the text
    f = re.sub("full-flop", '3', e)
    # Replace "no-flip" with '3' in the text
    g = re.sub("no-flip", '3', f)
    # Replace "half-flip" with '3' in the text
    h = re.sub("half-flip", '3', g)
    # Replace "pants-fire" with '1' in the text
    label = re.sub("pants-fire", '1', h)
    # Return the transformed text
    return label

# Convert the 'Label' column in the DataFrame df to string type
df['Label'] = df['Label'].astype(str)

# Apply the Fake_label function to each value in the 'Label' column and assign the result back to the 'Label' column
df['Label'] = df['Label'].apply(Fake_label)

## <span style="color:blue;">Support Vector Machine<span/>

In [5]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

In [6]:
# Features (x): All columns except the 'Label' column
x = df.drop(['Clean_News_Headline', 'Clean_Source', 'Label'], axis=1)

# Target variable (y): 'Label' column
y = df['Label']

<span style="color:blue;">Split the data into training, validation, and testing sets. Try different proportions and justify the final choices.</span>

In [7]:
# First, split into training and the rest
svm_x_train, svm_x_temp, svm_y_train, svm_y_temp = train_test_split(x, y, test_size=0.2, random_state=42)
 
# Then, split the rest into validation and testing
svm_x_val, svm_x_test, svm_y_val, svm_y_test = train_test_split(svm_x_temp, svm_y_temp, test_size=0.5, random_state=42)

First, the dataset is split into two sets: a training set that contains 80% of the total data, and a residual set. Consistency is ensured by the random_state=42 argument, which guarantees the generation of the same random split each time. The remaining data is then split into testing and validation sets, each of which contains 10% of the original dataset. The code makes guarantee that model evaluation is consistent and repeatable by using random_state=42 for both splits. Important components of developing a machine learning model, such as efficient model training, hyperparameter tuning, and objective assessment, are made possible by this method.

<span style="color:blue;">Experiment with different values of the C parameter; try the linear, rbf (with different choices of gamma) and polynomial kernels (with different degrees); try both options for decision_function_shape. Keep your best two models.</span>

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
# Feature scaling for SVM (faster convergence, and more stable optimization.)
sc = StandardScaler()
# Scale the training data
svm_x_train = sc.fit_transform(svm_x_train)
# Scale the validation data
svm_x_val = sc.transform(svm_x_val)
# Scale the test data
svm_x_test = sc.transform(svm_x_test)

In [10]:
# Define parameter grid
svm_param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [0.1, 0.01, 0.001],
    'degree': [2, 3, 4],
    'decision_function_shape': ['ovo', 'ovr']
}

In [11]:
# Initializing the SVM classifier
svm_classifier = SVC()

# Grid search with cross-validation
svm_grid_search = GridSearchCV(estimator=svm_classifier, param_grid=svm_param_grid, cv=5, n_jobs=-1, verbose=1)
svm_grid_search.fit(svm_x_train, svm_y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:   20.2s
[Parallel(n_jobs=-1)]: Done 402 tasks      | elapsed:   48.8s
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed: 27.5min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100],
                         'decision_function_shape': ['ovo', 'ovr'],
                         'degree': [2, 3, 4], 'gamma': [0.1, 0.01, 0.001],
                         'kernel': ['linear', 'rbf', 'poly']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [12]:
# Retrieve the best parameters found by the grid search
svm_grid_search.best_params_

{'C': 100,
 'decision_function_shape': 'ovo',
 'degree': 2,
 'gamma': 0.01,
 'kernel': 'rbf'}

In [13]:
# Get the top two best models
svm_cv_results = svm_grid_search.cv_results_
# Sort the models based on mean test score and get the indices of the top two models
svm_top_2 = svm_cv_results['mean_test_score'].argsort()[-2:][::-1]

In [14]:
# Initialize a list to store the best models
best_svm_models = []

# Counter variable to keep track of models
model_counter = 1

In [15]:
# Print top two best models with their parameters and accuracy
for idx in svm_top_2:
    # Extract parameters and accuracy for the current model
    model_params = svm_cv_results['params'][idx]
    model_accuracy = svm_cv_results['mean_test_score'][idx]
    # Initialize SVM classifier with best parameters
    best_svm_model = SVC(**model_params, random_state=42)
    # Train the best model
    best_svm_model.fit(svm_x_train, svm_y_train)
    # Store the best model in the list
    best_svm_models.append(best_svm_model)
    # Print model details
    print(f"Model {model_counter}:")
    print(f"Parameters: {model_params}")
    print(f"Accuracy: {model_accuracy}")
    print("-----------------------------")
    # Increment the model counter
    model_counter += 1

Model 1:
Parameters: {'C': 100, 'decision_function_shape': 'ovo', 'degree': 4, 'gamma': 0.01, 'kernel': 'rbf'}
Accuracy: 0.2532643206122142
-----------------------------
Model 2:
Parameters: {'C': 100, 'decision_function_shape': 'ovr', 'degree': 4, 'gamma': 0.01, 'kernel': 'rbf'}
Accuracy: 0.2532643206122142
-----------------------------


<span style="color:blue;">Use the classification_report and confusion_matrix functions in sklearn to display metrics for your best models, using training and validation (but not testing) data.</span>

In [16]:
# Function to display classification report and confusion matrix
def display_metrics(model, x, y, y_pred):
    # Classification report
    print("Classification Report:")
    print(classification_report(y, y_pred))
    # Confusion matrix
    print("\nConfusion Matrix:")
    print(confusion_matrix(y, y_pred))

In [17]:
# Get predictions for best model 1 using training data
svm_y_pred_train_1 = best_svm_models[0].predict(svm_x_train)
# Get predictions for best model 1 using validation data
svm_y_pred_val_1 = best_svm_models[0].predict(svm_x_val)

In [18]:
# Display metrics for the best model 1 using training data
print("Metrics for Best Model 1 using Training Data:")
display_metrics(best_svm_models[0], svm_x_train, svm_y_train, svm_y_pred_train_1)

# Display metrics for the best model 1 using validation data
print("\nMetrics for Best Model 1 using Validation Data:")
display_metrics(best_svm_models[0], svm_x_val, svm_y_val, svm_y_pred_val_1)

Metrics for Best Model 1 using Training Data:
Classification Report:
              precision    recall  f1-score   support

           1       0.51      0.31      0.38      1126
           2       0.23      0.85      0.36      1785
           3       0.30      0.09      0.13      1461
           4       0.28      0.01      0.02      1341
           5       0.21      0.03      0.05      1417
           6       0.00      0.00      0.00       838

    accuracy                           0.26      7968
   macro avg       0.25      0.21      0.16      7968
weighted avg       0.26      0.26      0.17      7968


Confusion Matrix:
[[ 345  730   30    4   17    0]
 [ 156 1516   67   11   35    0]
 [  70 1225  126    7   33    0]
 [  49 1149   84   11   48    0]
 [  42 1254   75    5   41    0]
 [  19  751   44    1   23    0]]

Metrics for Best Model 1 using Validation Data:
Classification Report:
              precision    recall  f1-score   support

           1       0.52      0.36      0.43

In [19]:
# Get predictions for best model 2 using training data
svm_y_pred_train_2 = best_svm_models[1].predict(svm_x_train)
# Get predictions for best model 2 using validation data
svm_y_pred_val_2 = best_svm_models[1].predict(svm_x_val)

In [20]:
# Display metrics for the best model 2 using training data
print("Metrics for Best Model 2 using Training Data:")
display_metrics(best_svm_models[1], svm_x_train, svm_y_train, svm_y_pred_train_2)

# Display metrics for the best model 2 using validation data
print("\nMetrics for Best Model 2 using Validation Data:")
display_metrics(best_svm_models[1], svm_x_val, svm_y_val, svm_y_pred_val_2)

Metrics for Best Model 2 using Training Data:
Classification Report:
              precision    recall  f1-score   support

           1       0.51      0.31      0.38      1126
           2       0.23      0.85      0.36      1785
           3       0.30      0.09      0.13      1461
           4       0.28      0.01      0.02      1341
           5       0.21      0.03      0.05      1417
           6       0.00      0.00      0.00       838

    accuracy                           0.26      7968
   macro avg       0.25      0.21      0.16      7968
weighted avg       0.26      0.26      0.17      7968


Confusion Matrix:
[[ 345  730   30    4   17    0]
 [ 156 1516   67   11   35    0]
 [  70 1225  126    7   33    0]
 [  49 1149   84   11   48    0]
 [  42 1254   75    5   41    0]
 [  19  751   44    1   23    0]]

Metrics for Best Model 2 using Validation Data:
Classification Report:
              precision    recall  f1-score   support

           1       0.52      0.36      0.43

<span style="color:blue;">Select and justify your final choice of hyperparameters based on the training and validation metrics. Provide a written analysis in markdown.</span>

Given that both models have the same hyperparameters and perform similarly across all evaluation metrics, the final choice between Model 1 and Model 2 comes down to other factors such as computational efficiency, model interpretability.

In this case, since I have chosen Model 1 as the final choice, it could be for reasons beyond the provided metrics. It might be because Model 1 offers simpler interpretation, easier implementation, or possibly it was chosen arbitrarily. Without additional context or reasoning provided, it's challenging to justify the choice based solely on the given information. However, it's reasonable to assume that Model 1 was selected due to its identical performance with Model 2, and other considerations such as simplicity or ease of use might have played a role in the decision-making process.

## Comparison

<span style="color:blue;">Provide a written comparison of the training and validation metrics for the best SVM and decision tree models using markdown. Select the best model and justify your choice.</span>

Upon examining the training and validation metrics for both the SVM and decision tree models, it's evident that both models perform comparably across various evaluation criteria. However, upon closer inspection, the SVM model exhibits slightly superior precision and recall for specific classes, notably class 4, in both the training and validation datasets. This nuanced improvement suggests that the SVM model may offer a more refined understanding of the data's underlying patterns, particularly in distinguishing class 4 instances.

I decide that the SVM model is the better option in this case since I prefer it. Although the overall performance of both models is similar, the SVM's marginally better performance in important measures, such precision and recall for particular classes, gives confidence in its capacity to correctly categorize data examples. Consequently, the SVM model is clearly the better choice, which is consistent with my focus on optimizing both predictive accuracy and model resilience.

<span style="color:blue;">Use your selected model to make predictions on the test set. Use the classification_report and confusion_matrix functions in sklearn to display metrics for the test data.</span>

In [26]:
# Make predictions on the test set
svm_y_pred_test = best_svm_models[0].predict(svm_x_test)

# Display classification report
print("Classification Report:")
print(classification_report(svm_y_test, svm_y_pred_test))

# Display confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(svm_y_test, svm_y_pred_test))

Classification Report:
              precision    recall  f1-score   support

           1       0.55      0.34      0.42       141
           2       0.28      0.90      0.42       256
           3       0.18      0.05      0.07       193
           4       0.25      0.01      0.01       157
           5       0.33      0.05      0.08       145
           6       0.00      0.00      0.00       104

    accuracy                           0.30       996
   macro avg       0.26      0.22      0.17       996
weighted avg       0.27      0.30      0.20       996

Confusion Matrix:
[[ 48  83   7   1   2   0]
 [ 14 231   7   1   3   0]
 [ 10 171   9   1   2   0]
 [  5 129  18   1   4   0]
 [ 10 124   4   0   7   0]
 [  1  94   6   0   3   0]]


<span style="color:blue;">Use markdown to review the best model; restate important metrics and describe how well it will work for your use case.</span>

Based on the classification report provided, the model's performance seems inadequate for our use case in sentiment analysis of product reviews. The metrics indicate low precision, recall, and F1-scores for most classes, suggesting that the model struggles to accurately classify reviews across different sentiments. While it shows relatively higher recall for some classes like 'barely-true', indicating better identification of certain sentiments, overall, its performance falls short of our requirements.

When it comes to identifying incorrect or misleading news or misinformation, the model's performance falls short of what is required. To successfully identify and classify news reviews based on their feelings, a model with improved accuracy, precision, recall, and F1-scores across all sentiment classes is necessary. The shortcomings evident in this model underscore the requirement for further optimization or exploration of alternative models to achieve the desired level of performance and reliability.